## quick example: each genealogy in an msprime simulation with recombination is one branch move away from each of its neighbors

In [1]:
import numpy as np
import strange
import toytree

In [2]:
rtr = toytree.rtree().coaltree(ntips=8,seed=42)
rtr3 = rtr.mod.node_scale_root_height(3)
rtr3.draw();

r7 r2 r1 r4 r3 r0 r5 r6

In [3]:
Ne = 50000
mut = 1e-8
kwargs = {
    "workdir": "../tests",
    "mutation_rate": mut,
    "recombination_rate": 1e-9,
    "theta": Ne*mut*4,
    "length": int(1e6), 
    "get_sequences": False,
    "random_seed": 42,
}

# simulation object
coal8 = strange.Coalseq(tree=rtr3, name="coal8", **kwargs)

In [4]:
len(coal8.tree_table)

1117

### ^1117 genealogies (a lot of ILS because of largeish pop size & shortish branches)

### I want a function that will tell me the heights of internal nodes on a given tree:

In [5]:
def get_int_node_vals(ttree):
    nodecoords = np.round(ttree.get_node_coordinates()[:,0],decimals=4)
    nodecoords = np.abs(nodecoords[nodecoords != 0])
    return(nodecoords)

In [6]:
get_int_node_vals(toytree.tree(coal8.tree_table.mstree[0]))

array([108781.7321, 121266.1262, 182111.8093,  64174.0087, 304083.2916,
       204293.3243, 473009.5932])

### How many internal node heights change from one tree to the next? This should indicate how many nodes are changing between trees.

In [7]:
nodes1 = np.round(get_int_node_vals(toytree.tree(coal8.tree_table.mstree[0])),decimals=4)
nodes2 = np.round(get_int_node_vals(toytree.tree(coal8.tree_table.mstree[1])),decimals=4)

counter = 0
for i in nodes1:
    if i in nodes2:
        counter += 1
        
counter

6

### This is saying that the exact heights for 6 of the 7 nodes stays the exact same -- implying that only one node changed from genealogy 0 to genealogy 1.


### let's iterate through every neighboring pair of trees to see how many nodes move between them:

In [8]:
# this time our counter is an array of zeros, where each entry corresponds to the 
# number of shared nodes between a neighboring pair of genealogies
counter = np.zeros((len(coal8.tree_table)-1),dtype=np.int8)
for treenum in range(len(coal8.tree_table)-1):
    nodes1 = np.round(get_int_node_vals(toytree.tree(coal8.tree_table.mstree[treenum])),decimals=4)
    nodes2 = np.round(get_int_node_vals(toytree.tree(coal8.tree_table.mstree[treenum+1])),decimals=4)
    for i in nodes1:
        if i in nodes2:
            counter[treenum] += 1

In [9]:
counter

array([6, 6, 6, ..., 6, 6, 6], dtype=int8)

In [10]:
np.all(counter == 6)

True

### this is telling us that every genealogy shares 6 of its 7 nodes with each of its two neighbors.

-- as a side note, McVean and Cardin 2005 (https://royalsocietypublishing.org/doi/full/10.1098/rstb.2005.1673) suggests that the length of a fragment corresponding to each genealogy is a function of the total branch length of that genealogy and the population recombination rate: "The distance along the unit interval until the first recombination event is exponentially distributed with rate ρ$T_{0}$/2." This might be useful for helping determine an expectation in number of base pairs as to how long a particular topology might be preserved.

### as proof of concept, here are the first 50 pairs of neighboring genealogies:

(you can see that a single branch is erased and reconnected somewhere else from one tree to the next)

In [11]:
for treenum in range(50):
    trees = toytree.mtree([toytree.tree(coal8.tree_table.mstree[treenum]),
                           toytree.tree(coal8.tree_table.mstree[treenum+1])])
    canvas, axes = trees.draw_tree_grid(shared_axis=True, orient='down', height=250, width=700)
    axes.y.domain.min = -1

3 1 2 5 4 7 6 8 3 1 2 5 4 7 6 8 0.0 118252.3 236504.5 354756.8 473009.1

3 1 2 5 4 7 6 8 3 1 2 5 4 7 6 8 0.0 118252.3 236504.5 354756.8 473009.1

3 1 2 5 4 7 6 8 3 1 2 5 4 7 6 8 0.0 118252.5 236505.0 354757.5 473010.0

3 1 2 5 4 7 6 8 3 1 2 5 4 7 6 8 0.0 118252.5 236505.0 354757.5 473010.0

3 1 2 5 4 7 6 8 3 1 2 5 4 7 6 8 0.0 136536.3 273072.5 409608.8 546145.1

...

2 1 3 5 4 8 6 7 2 1 3 5 4 8 6 7 0.0 82889.0 165778.0 248667.0 331556.0

2 1 3 5 4 8 6 7 1 3 2 5 4 8 6 7 0.0 82889.1 165778.1 248667.2 331556.2

1 3 2 5 4 8 6 7 2 1 3 5 4 8 6 7 0.0 82889.1 165778.1 248667.2 331556.2

2 1 3 5 4 8 6 7 2 1 3 5 4 8 6 7 0.0 82889.1 165778.1 248667.2 331556.2

2 1 3 5 4 8 6 7 2 1 3 5 4 8 6 7 0.0 82889.1 165778.1 248667.2 331556.2